# Prompt Engineering Lifecycle

### Setup

### Log a trace

In [2]:
from app import langsmith_rag

question = "Trace setter question?"
langsmith_rag(question)

USER_AGENT environment variable not set, consider setting it to identify your requests.


"I'm not sure what you're asking. Could you please provide more context or clarify your question about trace setters?"

We're going to pretty much define the same RAG application as before - with one crucial improvement.

Instead of pulling our `RAG_PROMPT` from utils.py, we're going to connect to the Prompt Hub in LangSmith.

Let's add the code snippet that will pull down our prompt that we just iterated on!

In [6]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from langsmith.client import convert_prompt_to_openai_format
from openai import OpenAI
from typing import List
import nest_asyncio

MODEL_NAME = "gpt-4o-mini"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0

# TODO: Remove this hard-coded prompt and replace it with Prompt Hub
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    # TODO: Let's use our prompt pulled from Prompt Hub instead of manually formatting here!
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    # formatted_prompt = prompt.invoke({"context":formatted_docs, "question": question})
    # messages = convert_prompt_to_openai_format(formatted_prompt)["messages"]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [8]:
question = "How do I set up tracing to LangSmith with @traceable?"
langsmith_rag(question)

"To set up tracing to LangSmith with the @traceable decorator in Python, decorate any function you want to trace with @traceable. Ensure that the LANGSMITH_TRACING environment variable is set to 'true' and provide your LANGSMITH_API_KEY in the environment. After this setup, you can start logging traces by executing your decorated functions."

In [2]:
# Prompt Hub Lifecycle Demo: Pull, Modify, Push

from langsmith import Client
import os

# Initialize client (uses LANGSMITH_API_KEY from environment)
client = Client()

# Step 1: Pull existing prompt from hub
print("Step 1: Pulling prompt from hub...")
prompt_identifier = "nik-hil:a7fc7883"  # Your existing prompt
pulled_prompt = client.pull_prompt(prompt_identifier, include_model=True)
print(f"Successfully pulled prompt: {prompt_identifier}")
print(f"Current prompt content:\n{pulled_prompt}\n")

# Step 2: Create an improved version
print("Step 2: Creating improved version of the prompt...")
improved_prompt = """You are a technical assistant specializing in LangSmith.
Answer questions using ONLY the provided context.
Be concise and direct - maximum 2 sentences.
If the context doesn't contain the answer, say "I don't have that information in the provided context."

Context: {context}
Question: {question}"""

print(f"Improved prompt:\n{improved_prompt}\n")

# Step 3: Push the improved version to hub
print("Step 3: Pushing improved prompt to hub...")
try:
    # This will create a new version of your prompt
    new_prompt = client.push_prompt(
        prompt_identifier.split(':')[0],  # Uses the same prompt name
        improved_prompt
    )
    print(f"Successfully pushed improved prompt!")
    print(f"New version can be accessed with the same identifier")
except Exception as e:
    print(f"Error pushing prompt: {e}")
    print("Note: Make sure you have write permissions for this prompt")

# Step 4: Show how to use it in the application
print("\nStep 4: Usage in your RAG application:")
print("""
Replace the hardcoded RAG_SYSTEM_PROMPT in generate_response with:

formatted_prompt = prompt.invoke({"context": formatted_docs, "question": question})
messages = convert_prompt_to_openai_format(formatted_prompt)["messages"]

""")

Step 1: Pulling prompt from hub...
Successfully pulled prompt: nik-hil:a7fc7883
Current prompt content:
first=ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'nik-hil', 'lc_hub_commit_hash': 'a7fc788350b817e97b4372f7057aab64687cec45b61d421632b439f9cc249aed'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the latest question in the conversation. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\n"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Context: Trace query syntax - Docs by LangChainSkip to main contentOur new LangChain Academy course on Deep Agents is now